## Altmap Experiments
### Compare altmap to map eq using networkx

In [3]:
from matplotlib.ticker import (AutoMinorLocator)

from altmap.altmap_helpers.general import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [103]:
# compute essential cost function values for a barbell network with
# given clique size
def ring_of_cliques_cost(clique_size = 3, number_of_cliques = 5, print_output=False):
    nc = clique_size
    c = number_of_cliques
    
    m = (nc*(nc-1) + 2) * c / 2.0 # number of edges in the network
    p0 = (nc - 1) /(2*m) # stat prob for 'normal' nodes
    pc = nc / (2*m) # stat prob for the connecting nodes

    # initial cost
    J_init = c*(nc-1)*p0*np.log2(1.0-p0) + c*pc*np.log2(1.0-pc)
    
    # maximum independent sets cost
    p1 = c*p0; p2 = c*pc
    J_ind = (nc-2)*p1*np.log2(1.0-p1) + 2*p2*np.log2(1.0-p2)
    
    # ground truth cost
    pi = (nc-2)*p0+2*pc; pnoti=1.0-pi; pinoti=1.0/m; pii = pi - pinoti
    J_true = -c*(pii*np.log2(pii/pi**2) + pinoti*np.log2(pinoti/(pi*pnoti)))
    
    if print_output:
        print (f"\nRing of {c} cliques with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
    
    return J_init, J_true, J_ind

num_cliques = 64
nc_max = 30
nc_list = list(range(3, nc_max + 1))
J_init_list = np.zeros((len(nc_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_list = np.zeros_like(J_init_list)
J_altmap_list = np.zeros_like(J_init_list)
J_altmap_sci_list = np.zeros_like(J_init_list)
for i,nc in enumerate(nc_list):
    J_init_list[i], J_true_list[i], J_ind_list[i] = ring_of_cliques_cost(nc, number_of_cliques=num_cliques)
    
    # compute actual objectives achieved by altmap
    # G = nx.ring_of_cliques(num_cliques, nc)
    # G = nx.convert_node_labels_to_integers(G, first_label=1)
    # communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True)
    # J_altmap_list[i] = altmap_cost(G, communities_found)
    # 
    # communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True, init='sc', update_inputfile=False)
    # J_altmap_sci_list[i] = altmap_cost(G, communities_found)

In [104]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# ax.suptitle(f'Ring of {num_cliques} cliques - objective over clique size')
ax.plot(nc_list, -J_true_list, 'x--', label='Ground truth')
ax.plot(nc_list, -J_init_list, '^--', label='Each node as a module')
ax.plot(nc_list, -J_ind_list, 'o--', label='Maximum independet sets')
ax.plot(nc_list, -J_altmap_list, '.--', ms=8, lw=2, label='Implementation')
ax.plot(nc_list, -J_altmap_sci_list, '.--', ms=8, lw=2, label='Implementation with SCI')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$', labelpad=20)
ax.set_ylabel('Synthesizing Infomap objective $\mathcal{J}(m)$', labelpad=20)
ax.legend()
plt.tight_layout()

In [108]:
num_cliques = 32
clique_size = 3
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# run community detection
# communities_found, num_communities_found,_,_ = infomap(G, altmap=False)
communities_found, num_communities_found,_,_ = infomap(G, altmap=True)
# communities_found, num_communities_found,_,_ = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

# cost = altmap_cost(G, communities_found)
# print (f'Achieved cost L = {cost}')


OrderedDict([(1, 1), (2, 1), (3, 1), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 16), (11, 16), (12, 16), (13, 3), (14, 3), (15, 3), (16, 3), (17, 3), (18, 3), (19, 4), (20, 4), (21, 4), (22, 4), (23, 4), (24, 4), (25, 5), (26, 5), (27, 5), (28, 5), (29, 5), (30, 5), (31, 6), (32, 6), (33, 6), (34, 6), (35, 6), (36, 6), (37, 7), (38, 7), (39, 7), (40, 7), (41, 7), (42, 7), (43, 8), (44, 8), (45, 8), (46, 8), (47, 8), (48, 8), (49, 9), (50, 9), (51, 9), (52, 9), (53, 9), (54, 9), (55, 17), (56, 17), (57, 17), (58, 10), (59, 10), (60, 10), (61, 10), (62, 10), (63, 10), (64, 11), (65, 11), (66, 11), (67, 11), (68, 11), (69, 11), (70, 12), (71, 12), (72, 12), (73, 12), (74, 12), (75, 12), (76, 13), (77, 13), (78, 13), (79, 13), (80, 13), (81, 13), (82, 14), (83, 14), (84, 14), (85, 14), (86, 14), (87, 14), (88, 15), (89, 15), (90, 15), (91, 15), (92, 15), (93, 15), (94, 1), (95, 1), (96, 1)])
We found 17 communities.


In [109]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)
plt.tight_layout()

In [29]:
# generate network
num_cliques = 4
clique_size = 3
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * clique_size
for c in range(2, num_cliques+1):
    labels += [c]*clique_size
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = list(range(1,clique_size+1)) * num_cliques
communities_ind = dict(zip(nodes_ids, labels))


plt.close('all')

plt.figure(figsize=(5,8))
drawNetwork(G, communities_true)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_init)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_ind)
plt.tight_layout()

# plt.close('all')
# fig, axs = plt.subplots(1,3)
# fig.suptitle(f'Sample ring of cliques networks')
# 
# drawNetwork(G, communities_true, ax=axs[0])
# axs[0].set_xlabel('Ground truth')
# drawNetwork(G, communities_init, ax=axs[1])
# axs[1].set_xlabel('Initial partition')
# drawNetwork(G, communities_ind, ax=axs[2])
# axs[2].set_xlabel('Maximum independent sets')